In [1]:
using Pkg
Pkg.activate(".")
using TrypColonies

  Activating project at `c:\Users\ank10ki\Dropbox\phd\TrypColonies\TrypColonies`


In [2]:
# julia --project -O3 --check-bounds=no diffusion_2D_expl_xpu.jl

using ParallelStencil
using ParallelStencil.FiniteDifferences2D

@init_parallel_stencil(Threads, Float64, 2)

using Printf
using GLMakie
using Parameters

@parallel function compute_flux!(qTx, qTy, T, Lam, dx, dy)
    @all(qTx) = -@av_xi(Lam)*@d_xi(T)/dx
    @all(qTy) = -@av_yi(Lam)*@d_yi(T)/dy
    return
end

@parallel function compute_update!(T, qTx, qTy, dt, dx, dy, decay_rate)
    @inn(T) = @inn(T) - dt*(@d_xa(qTx)/dx + @d_ya(qTy)/dy) - dt*decay_rate*@inn(T)
    return
end

compute_update! (generic function with 1 method)

In [4]:
function scale_time_step(dt_diff,dt_walker)
    if dt_diff >= dt_walker
        dt_diff = dt_walker
    else
        if dt_walker % dt_diff < 0.00000001 
            dt_diff = dt_walker/(dt_walker÷dt_diff)
        else
            dt_diff = dt_walker/(dt_walker÷dt_diff +1)
        end
    end
    return dt_diff, dt_walker
end

scale_time_step (generic function with 1 method)

In [6]:
@h @av_xi

`@av_xi(A)`: Compute averages between adjacent elements of `A` along the dimension x and select the inner elements of `A` in the remaining dimension. Corresponds to `@inn_y(@av_xa(A))`.


In [5]:
@with_kw struct parameters_physical
    N::Tuple{Int,Int}                   = (1000,1000)
    walker_speed_phy::Float64           = 5*10^-6
    L::Tuple{Float64,Float64}           = (0.04,0.04)
    scale_fac::Int                      = 1
    Diff_coe::Float64                   = 7*10^-10
    total_time::Float64                 = 200.0
    growth_rate::Float64                = 1.157*10^-5
    walker_step_size::Int               = 3
    dx::Float64                         = L[1]/N[1]
    dy::Float64                         = L[2]/N[2]
    xc::LinRange{Float64, Int64}        = LinRange(0.0, L[1], N[1])
    yc::LinRange{Float64, Int64}        = LinRange(0.0, L[2], N[2])
    Δt_diff_min::Float64                = minimum((L./(N)))^2/(4*Diff_coe)
    Δt_walker_min::Float64              = (minimum((L./(N)))*walker_step_size)/walker_speed_phy   
    Δt_diff::Float64                    = scale_time_step(Δt_diff_min, Δt_walker_min)[1]
    Δt_walker::Float64                  = scale_time_step(Δt_diff_min, Δt_walker_min)[2]
    time_steps_to_compute::Int          = round(Int,total_time/(minimum([Δt_diff,Δt_walker])))
end

parameters_physical

In [6]:
hans = parameters_physical()

parameters_physical
  N: Tuple{Int64, Int64}
  walker_speed_phy: Float64 4.9999999999999996e-6
  L: Tuple{Float64, Float64}
  scale_fac: Int64 1
  Diff_coe: Float64 7.000000000000001e-10
  total_time: Float64 200.0
  growth_rate: Float64 1.1570000000000001e-5
  walker_step_size: Int64 3
  dx: Float64 4.0e-5
  dy: Float64 4.0e-5
  xc: LinRange{Float64, Int64}
  yc: LinRange{Float64, Int64}
  Δt_diff_min: Float64 0.5714285714285715
  Δt_walker_min: Float64 24.000000000000007
  Δt_diff: Float64 0.5714285714285716
  Δt_walker: Float64 24.000000000000007
  time_steps_to_compute: Int64 350


In [7]:
hans.xc'

1×1000 adjoint(::LinRange{Float64, Int64}) with eltype Float64:
 0.0  4.004e-5  8.00801e-5  …  0.0398799  0.0399199  0.03996  0.04

In [8]:
ha = ones(Int, hans.N)   
TrypColonies.create_circle(ha, relative_size = 0.4)
ha = (ha.-1)*-1

UndefVarError: UndefVarError: `create_circle` not defined in `TrypColonies`
Suggestion: check for spelling errors or missing imports.

In [9]:
heatmap(ha)

In [58]:
@views function diffusion_2D_simplified(; do_visu=false, para_ph::parameters_physical = parameters_physical())
    
    results = Vector{Tuple{Array{Float64,2},Array{Float64,2},Array{Float64,2}}}(undef, 1)

    nvis = 20
    
    T = ones(Int, hans.N)   
    TrypColonies.create_circle!(T, relative_size = 0.4)
    T = Float64.((T.-1)*-1)

    qTx      = @zeros(para_ph.N[1]-1,para_ph.N[2]-2)
    qTy      = @zeros(para_ph.N[1]-2,para_ph.N[2]-1)

    Lam      = para_ph.Diff_coe .* ones(para_ph.N)
    dt       = para_ph.Δt_diff
    decay_rate = 0
    time=0.0; it=0; t_tic=0.0; niter=0

    timstep_walker = para_ph.Δt_walker
    timer_walker = 0.0
    adsorption_rate = 0.0    
    pos = rand([1:para_ph.N[1];], 2)
    # Time loop

    if do_visu
        time2 = Observable(0.0)
        fig = Figure()
    
        ax = Axis(fig[1, 1], title =  @lift("Timestep = $(round($(time2), sigdigits = 4))"))   #@lift("Timestep = $(round($(fig_obs.Time), digits = 1))")
        hm = heatmap!(ax, para_ph.xc, para_ph.yc, T, colorrange = (0.0, 1.0))

        color = Colorbar(fig[:, 2], hm)
        display(fig)
    end

    while time < para_ph.total_time
        it += 1

        if (it == 11) t_tic = Base.time(); niter = 0 end
        @parallel compute_flux!(qTx, qTy, T, Lam, para_ph.dx, para_ph.dy)
        @parallel compute_update!(T, qTx, qTy, dt, para_ph.dx, para_ph.dy, decay_rate)
        
        if timer_walker ÷ timstep_walker > 0 
            T[pos[1], pos[2]] += adsorption_rate
            pos += rand([-1, 0, 1], 2)
            
            if pos[1] < 1 pos[1] = para_ph.N[1] end
            if pos[1] > para_ph.N[1] pos[1] = 1 end
            if pos[2] < 1 pos[2] = para_ph.N[2] end
            if pos[2] > para_ph.N[2] pos[2] = 1 end
            timer_walker = 0.0
        end
        
        niter += 1
        timer_walker += timstep_walker
        time += dt
        
        if do_visu && (it % nvis == 0)
            delete!(ax, hm)
            delete!(color)
            time2[] = time
            hm = heatmap!(ax, para_ph.xc, para_ph.yc, T, colorrange = (0.0, 1.0))   
            color =Colorbar(fig[:, 2], hm)
            display(fig)
            sleep(0.000001)
            isopen(fig.scene) || break 
        end
    end
    t_toc = Base.time() - t_tic
    @printf("Computed %d steps, physical time = %1.3f\n", it, time)
    A_eff = 4/1e9*para_ph.N[1]*para_ph.N[2]*sizeof(Float64)  # Effective main memory access per iteration [GB]
    t_it  = t_toc/niter                  # Execution time per iteration [s]
    T_eff = A_eff/t_it                   # Effective memory throughput [GB/s]
    @printf("Perf: time = %1.3f sec, T_eff = %1.2f GB/s\n", t_toc, round(T_eff, sigdigits=3))
    results = (T, qTx, qTy)

    
    return results
end


diffusion_2D_simplified (generic function with 1 method)

In [60]:
para_phys = parameters_physical(total_time = 6000)
T, qtx, qty = diffusion_2D_simplified(do_visu = true, para_ph = para_phys)

Computed 10501 steps, physical time = 6000.571
Perf: time = 89.309 sec, T_eff = 3.76 GB/s


([0.0 0.0 … 0.0 0.0; 0.0 1.4717324315196347e-14 … 1.400828683652187e-14 0.0; … ; 0.0 1.400828683652187e-14 … 1.2859979445137964e-14 0.0; 0.0 0.0 … 0.0 0.0], [-2.6166824295486133e-19 -5.151063510318721e-19 … -4.902923890979668e-19 -2.4023003129500706e-19; -2.5343810807701084e-19 -5.361415755224046e-19 … -4.748448441567395e-19 -2.500576581635571e-19; … ; 2.500623578029597e-19 4.748495437961422e-19 … 4.685498714939246e-19 2.21442428112392e-19; 2.4023003129500706e-19 4.902876894585642e-19 … 4.500992805798287e-19 2.286568524674367e-19], [-2.6166824295486133e-19 -2.5343810807701084e-19 … 2.500623578029597e-19 2.4023003129500706e-19; -5.151063510318721e-19 -5.361415755224046e-19 … 4.748495437961422e-19 4.902876894585642e-19; … ; -4.902923890979668e-19 -4.748448441567395e-19 … 4.685498714939246e-19 4.500992805798287e-19; -2.4023003129500706e-19 -2.500576581635571e-19 … 2.21442428112392e-19 2.286568524674367e-19])

In [18]:
using ImageFiltering
using BenchmarkTools

In [49]:
para_ph = parameters_physical(N = (4000,4000))

T = ones(Int, para_ph.N)   
TrypColonies.create_circle!(T, relative_size = 0.4)
T = Float64.((T.-1)*-1)

qTx      = @zeros(para_ph.N[1]-1,para_ph.N[2]-2)
qTy      = @zeros(para_ph.N[1]-2,para_ph.N[2]-1)

Lam      = para_ph.Diff_coe .* ones(para_ph.N)
dt       = para_ph.Δt_diff
decay_rate = 0


timstep_walker = para_ph.Δt_walker



@parallel function compute_flux_simpi!(qTx, qTy, T, Lam, dx, dy)
    @all(qTx) = @d_xi(T)/dx
    @all(qTy) = @d_yi(T)/dy
    return
end

compute_flux_simpi! (generic function with 1 method)

In [54]:
@benchmark @parallel compute_flux!(qTx, qTy, T, Lam, para_ph.dx, para_ph.dy)

BenchmarkTools.Trial: 86 samples with 1 evaluation.
 Range (min … max):  57.229 ms …  61.326 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     57.833 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   58.201 ms ± 858.724 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃▆▃█▃▄█  ▁                                                 
  ▄▄▇███████▁▁█▄▆▆▄▄▄▆▁▄▆▆▆▆▄▄▁▁▄▁▄▁▇▁▄▄▄▁▁▁▁▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  57.2 ms         Histogram: frequency by time         61.2 ms <

 Memory estimate: 4.94 KiB, allocs estimate: 92.

In [53]:
@benchmark grad_x, grad_y = imgradients(T,KernelFactors.sobel )

BenchmarkTools.Trial: 13 samples with 1 evaluation.
 Range (min … max):  253.328 ms … 716.239 ms  ┊ GC (min … max):  4.79% … 64.67%
 Time  (median):     335.301 ms               ┊ GC (median):    27.70%
 Time  (mean ± σ):   424.932 ms ± 192.383 ms  ┊ GC (mean ± σ):  42.12% ± 23.28%

  ▁▁   █▁▁  █▁                                           ▁ ▁  █  
  ██▁▁▁███▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁█ ▁
  253 ms           Histogram: frequency by time          716 ms <

 Memory estimate: 488.73 MiB, allocs estimate: 1585.

In [51]:
grad_y

1000×1000 Matrix{Float64}:
 1.83967e-15  3.75446e-15  3.71152e-15  …  -3.44693e-15  -1.75104e-15
 7.43379e-15  1.48998e-14  1.49976e-14     -1.39287e-14  -6.94897e-15
 1.48998e-14  2.98651e-14  3.00601e-14     -2.79178e-14  -1.39286e-14
 2.24313e-14  4.49598e-14  4.52549e-14     -4.20298e-14  -2.09685e-14
 3.00599e-14  6.02522e-14  6.06454e-14     -5.63239e-14  -2.81008e-14
 3.78207e-14  7.58051e-14  7.63025e-14  …  -7.08657e-14  -3.53546e-14
 4.57448e-14  9.16909e-14  9.2289e-14      -8.57138e-14  -4.27639e-14
 5.38695e-14  1.07972e-13  1.0868e-13      -1.00938e-13  -5.03577e-14
 6.22263e-14  1.24726e-13  1.25539e-13     -1.16598e-13  -5.81723e-14
 7.08552e-14  1.42017e-13  1.42948e-13     -1.32767e-13  -6.62372e-14
 ⋮                                      ⋱                
 5.8184e-14   1.1662e-13   1.17385e-13     -1.09021e-13  -5.43899e-14
 5.03655e-14  1.00953e-13  1.01612e-13     -9.43702e-14  -4.70826e-14
 4.27688e-14  8.57228e-14  8.62856e-14     -8.01356e-14  -3.99793e-14
 3.53

In [52]:
qTy

3998×3999 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                 ⋮              
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [ ]:
Lx, Ly   = 10.0, 10.0    # domain extend
λ0       = 13.5           # background heat conductivity
ttot     = 0.5           # total time
# Numerics
n        = 2
nx, ny   = n*64, n*64    # number of grid points
ndt      = 10            # sparse timestep computation
nvis     = 50            # sparse visualisation
# Derived numerics
dx, dy   = Lx/nx, Ly/ny  # grid cell size
xc, yc   = LinRange(dx/2, Lx-dx/2, nx), LinRange(dy/2, Ly-dy/2, ny)
min_dxy2 = min(dx,dy)^2
# Array initialisation
T        = Data.Array(exp.(.-(xc .- Lx/2).^2 .-(yc' .- Ly/2).^2))
qTx      = @zeros(nx-1,ny-2)
qTy      = @zeros(nx-2,ny-1)
# keep following lines for the case of variable difffusion coefficient at border of colony
"""
ρCp      =   ones(nx  ,ny  )
ρCp[((xc.-Lx/3).^2 .+ (yc'.-Ly/3).^2).<1.0].=0.01
ρCp      = Data.Array(ρCp)
"""
Lam      = λ0 .+ 0.1.*@rand(nx,ny)
#dt       = min_dxy2/maximum(Lam./ρCp)/4.1
dt       = min_dxy2/maximum(Lam)/4.1
decay_rate = 0.1

0.1

In [7]:
@views function diffusion_2D(; do_visu=false)
    n_t = 1
    results = Vector{Tuple{Array{Float64,2},Array{Float64,2},Array{Float64,2}}}(undef, n_t)
    Threads.@threads for i in 1:n_t 

        # Physics
        Lx, Ly   = 10.0, 10.0    # domain extend
        λ0       = 55.5           # background heat conductivity
        ttot     = 0.5           # total time
        # Numerics
        n        = 2
        nx, ny   = n*128, n*128    # number of grid points
        ndt      = 10            # sparse timestep computation
        nvis     = 50            # sparse visualisation
        # Derived numerics
        dx, dy   = Lx/nx, Ly/ny  # grid cell size
        xc, yc   = LinRange(dx/2, Lx-dx/2, nx), LinRange(dy/2, Ly-dy/2, ny)
        min_dxy2 = min(dx,dy)^2
        # Array initialisation
        T        = Data.Array(exp.(.-(xc .- Lx/2).^2 .-(yc' .- Ly/2).^2))
        qTx      = @zeros(nx-1,ny-2)
        qTy      = @zeros(nx-2,ny-1)
        ρCp      =   ones(nx  ,ny  )
        ρCp[((xc.-Lx/3).^2 .+ (yc'.-Ly/3).^2).<1.0].=0.01
        ρCp      = Data.Array(ρCp)
        Lam      = λ0 .+ 0.1.*@rand(nx,ny)
        dt       = min_dxy2/maximum(Lam./ρCp)/4.1
        decay_rate = 0
        #dt = 0.000001
        time=0.0; it=0; t_tic=0.0; niter=0
        adsorption_rate = 0.1    

        pos = rand([1:nx;], 2)
        # Time loop
        if do_visu
            time2 = Observable(0.0)
            fig = Figure()
            #heatmap(t[2:end-1,2:end-1])
        
            ax = Axis(fig[1, 1], title =  @lift("Timestep = $(round($(time2), sigdigits = 4))"))   #@lift("Timestep = $(round($(fig_obs.Time), digits = 1))")
            hm = heatmap!(ax, xc[2:end-1], yc[2:end-1], Array(T)[2:end-1, 2:end-1],colorrange = (0.0, 1.0))

            strength = vec(sqrt.(qTx[2:end,:] .^ 2 .+ qTy[:,2:end].^ 2))
            #arrow = arrows!(ax, xc[2:end-1], yc[2:end-1], qTx[2:end,:], qTy[:,2:end])#,arrowsize =strength*10, lengthscale = maximum(strength))
            color = Colorbar(fig[:, 2], hm)
            display(fig)
        end
    

        while time < ttot
            it += 1
            if (it == 11) t_tic = Base.time(); niter = 0 end
            if (it % ndt == 0) dt = min_dxy2/maximum(Lam./ρCp)/4.1 end # done every ndt to improve perf
            @parallel compute_flux!(qTx, qTy, T, Lam, dx, dy)
            @parallel compute_update!(T, qTx, qTy, dt, dx, dy, decay_rate)
            
            T[pos[1], pos[2]] += adsorption_rate
            pos += rand([-1, 0, 1], 2)
            
            if pos[1] < 1 pos[1] = nx end
            if pos[1] > nx pos[1] = 1 end
            if pos[2] < 1 pos[2] = ny end
            if pos[2] > ny pos[2] = 1 end
            
            niter += 1
            time += dt
            
            if do_visu && (it % nvis == 0)
    
                delete!(ax, hm)
                #delete!(ax, arrow)
                delete!(color)
                time2[] = time
                hm = heatmap!(ax, xc[2:end-1], yc[2:end-1], Array(T)[2:end-1, 2:end-1],colorrange = (0.0, 1.0))
                strength = vec(sqrt.(qTx[2:end,:] .^ 2 .+ qTy[:,2:end].^ 2))
                #arrow = arrows!(ax, xc[2:end-1], yc[2:end-1], qTx[2:end,:], qTy[:,2:end],arrowsize =strength*10) #, lengthscale = maximum(strength))

                color =Colorbar(fig[:, 2], hm)
                display(fig)
                sleep(0.000001)
                isopen(fig.scene) || break 
            end
        end
        t_toc = Base.time() - t_tic
        @printf("Computed %d steps, physical time = %1.3f\n", it, time)
        A_eff = 4/1e9*nx*ny*sizeof(Float64)  # Effective main memory access per iteration [GB]
        t_it  = t_toc/niter                  # Execution time per iteration [s]
        T_eff = A_eff/t_it                   # Effective memory throughput [GB/s]
        @printf("Perf: time = %1.3f sec, T_eff = %1.2f GB/s\n", t_toc, round(T_eff, sigdigits=3))
        results[i] = (Array(T)', qTx, qTy)

    end
    return results
end


diffusion_2D (generic function with 1 method)

In [8]:
res = diffusion_2D(do_visu=true)

Computed 341050 steps, physical time = 0.023
Perf: time = 457.277 sec, T_eff = 1.56 GB/s


1-element Vector{Tuple{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}}:
 ([1.3 1.7000000000000004 … 0.5 0.30000000000000004; 0.7999999999999999 1.1605025548314647 … 0.5857527165908124 0.6; … ; 0.9999999999999999 0.9565062906728129 … 0.7111194689081805 0.5; 0.9999999999999999 1.2 … 1.2 0.8999999999999999], [-512.5839391930457 -478.6904105568363 … -389.82897889408133 61.81556116977608; 78.8653037768331 -28.203158006776995 … -177.72882058970643 6.037224094669315; … ; -32.231046373905336 -127.04201508659867 … -37.596299119471254 -208.86831530091686; -20.26973591538989 -455.46847200281934 … -28.045509829564498 300.3755376162225], [767.4198132860605 175.97036547966005 … -401.8512807397983 -346.09442614916577; 135.10303246219377 68.95226905464897 … -218.1962922275406 -210.1746431420457; … ; -89.75472827566637 -39.01869675865227 … -14.964718198699327 91.55810688674879; -121.93846920718424 -133.90231555804607 … -186.0846528569739 -695.3313000534282])

In [11]:
function scale_time_step(dt_diff,dt_walker)
    if dt_diff >= dt_walker
        dt_diff = dt_walker
    else
        if dt_walker % dt_diff < 0.00000001 
            dt_diff = dt_walker/(dt_walker÷dt_diff)
        else
            dt_diff = dt_walker/(dt_walker÷dt_diff +1)
        end
    end
    return dt_diff, dt_walker
end



scale_time_step (generic function with 1 method)

In [24]:
scale_time_step(1, 0.1)

(0.1, 0.1)

In [19]:
scale_time_step(3.5, 10)

(3.3333333333333335, 10)

In [25]:
@with_kw struct parameters_physical
    N_grad::Tuple{Int,Int}      = (400,400)
    walker_speed_phy::Float64   = 5*10^-6
    size_phy::Float64           = 0.04
    Diff_coe::Float64           = 7*10^-10
    Δt_diff_min::Float64        = (size_phy/N_grad[1])^2/(4*Diff_coe)
    walker_step_size::Int       = 4
    Δt_walker_min::Float64      = (size_phy/N_grad[1])/walker_speed_phy   
    Δt_diff::Float64            = scale_time_step(Δt_diff_min, Δt_walker_min)[1]
    Δt_walker::Float64          = scale_time_step(Δt_diff_min, Δt_walker_min)[2]
end

parameters_physical

In [54]:
        # Physics
        Lx, Ly   = 10.0, 10.0    # domain extend
        λ0       = 55.5           # background heat conductivity
        ttot     = 0.5           # total time
        # Numerics
        nx, ny   = 128, 128    # number of grid points
        nvis     = 50            # sparse visualisation
        # Derived numerics
        dx, dy   = Lx/nx, Ly/ny  # grid cell size
        xc, yc   = LinRange(dx/2, Lx-dx/2, nx), LinRange(dy/2, Ly-dy/2, ny)
        min_dxy2 = min(dx,dy)^2
        # Array initialisation
        T        = Data.Array(exp.(.-(xc .- Lx/2).^2 .-(yc' .- Ly/2).^2))
        qTx      = @zeros(nx-1,ny-2)
        qTy      = @zeros(nx-2,ny-1)

        Lam      = λ0 .+ 0.1.* rand(nx,ny)
        dt       = min_dxy2/maximum(Lam)/4.1
        decay_rate = 0
        time=0.0; it=0; t_tic=0.0; niter=0
        timstep_walker = 0.01
        timer_walker = 0.0
        adsorption_rate = 0.1    

0.1

In [51]:
ttot/dt

18674.48223609371

In [38]:
@views function diffusion_2D_arrows(; do_visu=false)
    n_t = 1
    results = Vector{Tuple{Array{Float64,2},Array{Float64,2},Array{Float64,2}}}(undef, n_t)
    Threads.@threads for i in 1:n_t 

        # Physics
        Lx, Ly   = 10.0, 10.0    # domain extend
        λ0       = .9           # background heat conductivity
        ttot     = 20.5           # total time
        # Numerics
        nx, ny   = 400, 400    # number of grid points
        nvis     = 20            # sparse visualisation
        # Derived numerics
        dx, dy   = Lx/nx, Ly/ny  # grid cell size
        xc, yc   = LinRange(dx/2, Lx-dx/2, nx), LinRange(dy/2, Ly-dy/2, ny)
        min_dxy2 = min(dx,dy)^2
        # Array initialisation
        T        = Data.Array(exp.(.-(xc .- Lx/2).^2 .-(yc' .- Ly/2).^2))
        qTx      = @zeros(nx-1,ny-2)
        qTy      = @zeros(nx-2,ny-1)

        Lam      = λ0 .+ 0.1.* rand(nx,ny)
        dt       = min_dxy2/maximum(Lam)/4.1
        decay_rate = 0
        time=0.0; it=0; t_tic=0.0; niter=0
        timstep_walker = 0.01
        timer_walker = 0.0
        adsorption_rate = 0.1    
        arrow_fac = 0.01
        pos = rand([1:nx;], 2)
        # Time loop
        if do_visu
            time2 = Observable(0.0)
            fig = Figure()
                    
            ax = Axis(fig[1, 1], title =  @lift("Timestep = $(round($(time2), sigdigits = 4))"))   #@lift("Timestep = $(round($(fig_obs.Time), digits = 1))")
            hm = heatmap!(ax, xc[2:end-1], yc[2:end-1], Array(T)[2:end-1, 2:end-1],colorrange = (0.0, 1.0))

            strength = vec(sqrt.(qTx[2:end,:] .^ 2 .+ qTy[:,2:end].^ 2))
            arrow = arrows!(ax, xc[2:end-1], yc[2:end-1], qTx[2:end,:].*arrow_fac, qTy[:,2:end].*arrow_fac)#,arrowsize =strength*10, lengthscale = maximum(strength))
            color = Colorbar(fig[:, 2], hm)
            display(fig)
        end
    

        while time < ttot
            it += 1
            if (it % ndt == 0) dt = min_dxy2/maximum(Lam)/4.1 end # done every ndt to improve perf

            if (it == 11) t_tic = Base.time(); niter = 0 end
            @parallel compute_flux!(qTx, qTy, T, Lam, dx, dy)
            @parallel compute_update!(T, qTx, qTy, dt, dx, dy, decay_rate)
            
            if timer_walker ÷ timstep_walker > 0 
                T[pos[1], pos[2]] += adsorption_rate
                pos += rand([-1, 0, 1], 2)
                
                if pos[1] < 1 pos[1] = nx end
                if pos[1] > nx pos[1] = 1 end
                if pos[2] < 1 pos[2] = ny end
                if pos[2] > ny pos[2] = 1 end
                timer_walker = 0.0
            end
            
            niter += 1
            timer_walker += timstep_walker
            time += dt
            
            if do_visu && (it % nvis == 0)
    
                delete!(ax, hm)
                delete!(ax, arrow)
                delete!(color)
                time2[] = time
                hm = heatmap!(ax, xc[2:end-1], yc[2:end-1], Array(T)[2:end-1, 2:end-1],colorrange = (0.0, 1.0))
                strength = vec(sqrt.(qTx[2:end,:] .^ 2 .+ qTy[:,2:end].^ 2))
                arrow = arrows!(ax, xc[2:end-1], yc[2:end-1], qTx[2:end,:].*arrow_fac, qTy[:,2:end].*arrow_fac,arrowsize =strength.*arrow_fac) #, lengthscale = maximum(strength))

                color =Colorbar(fig[:, 2], hm)
                display(fig)
                sleep(0.000001)
                isopen(fig.scene) || break 
            end
        end
        t_toc = Base.time() - t_tic
        @printf("Computed %d steps, physical time = %1.3f\n", it, time)
        A_eff = 4/1e9*nx*ny*sizeof(Float64)  # Effective main memory access per iteration [GB]
        t_it  = t_toc/niter                  # Execution time per iteration [s]
        T_eff = A_eff/t_it                   # Effective memory throughput [GB/s]
        @printf("Perf: time = %1.3f sec, T_eff = %1.2f GB/s\n", t_toc, round(T_eff, sigdigits=3))
        results[i] = (Array(T)', qTx, qTy)

    end
    return results
end


diffusion_2D_arrows (generic function with 1 method)

In [39]:
diffusion_2D_arrows(do_visu = true)

Computed 200 steps, physical time = 0.030
Perf: time = 3.199 sec, T_eff = 0.30 GB/s


1-element Vector{Tuple{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}}:
 ([2.475790021935655e-22 3.175006075445606e-22 … 3.175006075445606e-22 2.4757900219356377e-22; 3.175006075445606e-22 8.13315711782669e-21 … 8.277144484180174e-21 3.175006075445606e-22; … ; 3.175006075445606e-22 8.26300559180313e-21 … 8.139968963016474e-21 3.175006075445606e-22; 2.4757900219356377e-22 3.175006075445606e-22 … 3.175006075445606e-22 2.47579002193562e-22], [-2.83965911866338e-19 -5.944835538155809e-19 … -6.058402276334289e-19 -2.8937593165049694e-19; -2.992090200296785e-19 -6.098574626741246e-19 … -6.314035170787597e-19 -3.01711145428418e-19; … ; 3.0417600805571853e-19 6.350845889772453e-19 … 6.338737211999712e-19 2.939829732746634e-19; 2.962061448859994e-19 6.112609936475918e-19 … 6.040641379540087e-19 2.882665140356874e-19], [-2.79173651869069e-19 -2.9065920186536813e-19 … 3.0428093944506084e-19 2.8950419595164943e-19; -6.0138772494299405e-19 -6.185104334234374e-19 … 6.116948889519078e-19 5.803388

In [ ]:
grid1 = rand(100,100)*0.01

grid2 = rand(1000,1000)*2;

x_c = LinRange(0.0,10.0,100)
x_c2 = LinRange(0.0,10.0,1000)

1000-element LinRange{Float64, Int64}:
 0.0, 0.01001, 0.02002, 0.03003, 0.04004, …, 9.96997, 9.97998, 9.98999, 10.0

10000

In [50]:
fig = Figure()
ax1 = Axis(fig[1,1])
heatmap!(ax1,x_c,x_c,grid1,colormap = :hsv,alpha =0.5)
heatmap!(ax1,x_c2,x_c2,grid2, alpha =0.5)

Heatmap{Tuple{Vector{Float64}, Vector{Float64}, Matrix{Float32}}}

In [51]:
fig

UndefVarError: UndefVarError: `para_ph` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [54]:
grid1 = rand(Int64, 10,10)
grid2 = rand(10,10)

10×10 Matrix{Float64}:
 0.0678221  0.871221   0.107529  0.843102   …  0.59661    0.828477  0.510786
 0.752071   0.117868   0.924611  0.96143       0.71437    0.352089  0.852739
 0.139215   0.0791835  0.228044  0.0399396     0.385212   0.026549  0.377956
 0.0370868  0.616343   0.277536  0.854674      0.206505   0.639441  0.973214
 0.915905   0.119122   0.945597  0.273898      0.146555   0.162906  0.631072
 0.379745   0.651908   0.680479  0.478024   …  0.578919   0.110301  0.357015
 0.696242   0.87866    0.553523  0.528205      0.882497   0.579048  0.612779
 0.894975   0.992703   0.213082  0.804774      0.314418   0.737502  0.0415881
 0.470302   0.858333   0.670084  0.422986      0.0803419  0.57338   0.192121
 0.591976   0.967339   0.496276  0.876278      0.875539   0.523989  0.798951

In [59]:
hans = (grid1,grid2);

In [60]:
typeof(hans)

Tuple{Matrix{Int64}, Matrix{Float64}}

In [57]:
let t = 1

for i in 1:10 
    t += 1
end
t
end


11